In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import LabelEncoder
import holidays
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [3]:
dataset = pd.read_excel("영화 데이터.xlsx")
# 누적관객수, 스크린수, 상영횟수 1이하 데이터 제거 (없는 데이터가 많음)
dataset.drop(dataset[(dataset['누적관객수']<=1) & (dataset['스크린수']<=1) & (dataset['상영횟수']<=1)].index, inplace=True)
# 누적매출액이 0인 데이터 제거 (마찬가지로 없는 데이터가 많음)
dataset.drop(dataset[dataset['누적매출액'] == 0].index, inplace=True)
# 영화 개봉 후 발생하는 데이터 제거
dataset = dataset.drop(['순위', '매출액', '점유율', '관객수', '누적관객수', '스크린수', '상영횟수', '국적'], axis=1)

In [4]:
dataset.head(1)

,영화명,개봉일,누적매출액,대표국적,제작사,배급사,등급,장르,감독,배우
0,태극기 휘날리며,2004-02-05,15687180500,한국,강제규필름,"(주)다자인소프트,(주)쇼박스",15세관람가,"전쟁,드라마",강제규,"장동건,원빈,이은주,공형진,조원희,이장욱,이영란,정희준,박유신,김효열,김효준,최재혁..."


In [ ]:
# 모델 불러오기
models = []

for i in range(10):
    model = joblib.load(f'lgbm_models/lgbm_model{i}.pkl')
    models.append(model)

models

In [6]:
def startPredict(data):

    # 라벨 인코딩 데이터
    df = pd.read_csv("labelData.csv")

    # 예측할 데이터 전처리
    test = pd.DataFrame(dataset.iloc[0])
    test = test.T
    test = test.drop(['제작사'], axis=1)
    test = test.drop(['누적매출액'], axis=1)

    test['대표국적'][~test['대표국적'].isin(['한국', '미국', '일본', '프랑스', '영국'])] = '기타'

    test['등급'] = test['등급'].str.split(',').str[0]
    test['등급'][test['등급'].isin(['전체관람가', '연소자관람가', '미성년자관람가', '모든 관람객이 관람할 수 있는 등급', '기타', '미정'])] = 0
    test['등급'][test['등급'].isin(['12세관람가', '12세이상관람가', '중학생이상관람가', '12세 미만인 자는 관람할 수 없는 등급', '국민학생관람불가'])] = 12
    test['등급'][test['등급'].isin(['15세관람가', '15세이상관람가', '고등학생이상관람가', '15세 미만인 자는 관람할 수 없는 등급'])] = 15
    test['등급'][test['등급'].isin(['18세관람가', '청소년관람불가', '18세 미만인 자는 관람할 수 없는 등급', '미성년자관람불가', '연소자관람불가'])] = 18
    test['등급'].fillna(-1, inplace=True)
    test['등급'] = test['등급'].astype(int)

    for i in range(1, 4):
        test[f'배우{i}'] = test['배우'].str.split(',').str[i - 1]

    test = test.drop(['배우'], axis=1)

    genre = ["SF","가족","공연","공포(호러)","기타","다큐멘터리","드라마","멜로/로맨스","뮤지컬","미스터리","범죄","사극","서부극(웨스턴)","성인물(에로)","스릴러","애니메이션","액션","어드벤처","전쟁","코미디","판타지"]
    g = test['장르'].str.split(',')

    for i in genre:
        test[i] = test['장르'].apply(lambda x: i in x).astype(int)

    test = test.drop(['장르'], axis=1)

    test['감독'] = test['감독'].str.split(',').str[0]

    test['배급사'] = test['배급사'].str.split(',').str[0]

    test['개봉일'] = pd.to_datetime(test['개봉일'])
    test['년'] = test['개봉일'].dt.year
    test['월'] = test['개봉일'].dt.month
    test['일'] = test['개봉일'].dt.day
    test['요일'] = test['개봉일'].dt.dayofweek    # 0 = 월요일 ~

    kr_holidays = holidays.KR() 
    test['휴일'] = test['개봉일'].apply(lambda x: 1 if (x.weekday() in [5, 6] or x in kr_holidays) else 0)

    test = test.drop(['개봉일'], axis=1)

    categories = ['영화명', '대표국적', '배급사', '감독', '배우1', '배우2', '배우3']

    unique_values = {}
    for cat in categories:
        unique_values[cat] = df[cat].unique()

    # 새로운 데이터에 대해 라벨 인코딩 수행
    for cat in categories:
        for value in test[cat].unique():
            # 학습 데이터에서 찾지 못한 새로운 값인 경우에만 라벨 추가
            if value not in unique_values[cat]:
                unique_values[cat] = np.append(unique_values[cat], value)

    # 라벨 인코딩 적용
    for cat in categories:
        le = LabelEncoder()
        test[cat].fillna("NaN", inplace=True)
        le.fit(unique_values[cat])
        test[cat] = le.transform(test[cat])

    columns_order = ['영화명', '대표국적', '배급사', '등급', '감독', '배우1', '배우2', '배우3', 'SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로/로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)', '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지', '년', '월', '일', '요일', '휴일', '코로나']
    test = test.reindex(columns=columns_order)

    result = []
    for model in models:
        y_pred = model.predict(test.values.reshape(1, -1))
        print(y_pred)
        result.append(y_pred)

    print(np.mean(result))